<a href="https://colab.research.google.com/github/ARADHYA299/GoogleSolutionChallenge2025/blob/main/GenAIfinancialassistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q google-genai
!pip show google-genai
!pip install -q gradio

Name: google-genai
Version: 1.4.0
Summary: GenAI Python SDK
Home-page: https://github.com/googleapis/python-genai
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: Apache-2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: google-auth, httpx, pydantic, requests, typing-extensions, websockets
Required-by: 
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2

In [3]:
import gradio as gr
import requests
import time
import os

# Get API key from Hugging Face Spaces secret
HF_API_KEY = os.environ.get("tutorhfkey")
if not HF_API_KEY:
    print("Warning: API key 'tutorhfkey' not found in environment variables")
    # Fallback only for development
    HF_API_KEY = "YOUR_API_KEY_PLACEHOLDER"

HF_API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"
headers = {"Authorization": f"Bearer {HF_API_KEY}"}

# System instructions for the financial tutor
SYSTEM_INSTRUCTIONS = """You are a financial markets tutor and adviser designed to educate and recommend/advise beginners, intermediates or experts about investing, financial instruments, money markets and market dynamics.
Answer briefly and to the point, don't explain unnecessarily, and use things like real life examples or the things with which they can relate them.

Capabilities:
- Teach financial concepts in an interactive and engaging way.
- Guide users on different types of financial markets (stocks, bonds, crypto, commodities, etc.).
- Explain investment strategies, risk management, and portfolio diversification but tell these only when asked.
- Answer questions related to fundamental and technical analysis.
- If they give you certain budgets provide them the best areas to invest along with risk involved but warn them about the risks and also advise to think before investing.

Guidelines:
- Begin by understanding the user's financial knowledge level.
- If the user is new, first ask them which financial market they are interested in.
- Provide structured, step-by-step explanations with examples.
- Use simple language and try to give response in an example or things with which the user can relate."""

# Function to query the model with retry logic
def query_model(prompt, max_retries=3):
    """Send a request to the Hugging Face API with retry logic"""
    payload = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": 512,
            "temperature": 0.7,
            "top_p": 0.95
        }
    }

    for attempt in range(max_retries):
        try:
            response = requests.post(
                HF_API_URL,
                headers=headers,
                json=payload,
                timeout=30
            )

            # Handle successful response
            if response.status_code == 200:
                return response.json()[0]["generated_text"]

            # Handle model loading
            elif response.status_code == 503:
                wait_time = 2 * (attempt + 1)
                print(f"Model is loading. Waiting {wait_time} seconds...")
                time.sleep(wait_time)

                # If last attempt, return a friendly message
                if attempt == max_retries - 1:
                    return "The model is currently loading. Please try again in a moment."

            # Handle other errors
            else:
                print(f"Error: Received status code {response.status_code}")
                error_msg = f"Error: Received status code {response.status_code}. "
                if attempt == max_retries - 1:
                    return f"{error_msg} Please try again later."
                time.sleep(2)

        except Exception as e:
            print(f"Exception occurred: {str(e)}")
            if attempt == max_retries - 1:
                return f"Sorry, I encountered an error: {str(e)}. Please try again later."
            time.sleep(2)

    return "I'm having trouble connecting to the server. Please try again later."

# Format the conversation for the model
def format_prompt(messages):
    """Format the conversation history for the Mistral model"""
    prompt = f"<s>[INST] {SYSTEM_INSTRUCTIONS}\n\n"

    for i, message in enumerate(messages):
        role = message["role"]
        content = message["content"]

        if role == "user":
            if i == len(messages) - 1:
                # Last user message
                prompt += f"{content} [/INST]"
            else:
                # User message followed by assistant message
                assistant_msg = messages[i+1]["content"] if i+1 < len(messages) and messages[i+1]["role"] == "assistant" else ""
                prompt += f"{content} [/INST] {assistant_msg} </s>"
                if i+2 < len(messages) and messages[i+2]["role"] == "user":
                    prompt += "<s>[INST] "

    return prompt

# Chatbot logic
def respond(message, chat_history, conversation_state):
    """Process user message and get response from the model"""
    # Initialize conversation if empty
    if not conversation_state:
        conversation_state = [
            {"role": "assistant", "content": "Hello! I'm your financial markets tutor. Before we begin, could you tell me your current level of knowledge about financial markets? Are you a beginner, intermediate, or advanced?"}
        ]

    # Add user message to conversation
    conversation_state.append({"role": "user", "content": message})

    # Format the prompt and get response
    prompt = format_prompt(conversation_state)
    response_text = query_model(prompt)

    # Extract just the model's response part
    if "[/INST]" in response_text:
        response_text = response_text.split("[/INST]")[-1].strip()

    # Clean up response text (remove trailing conversation markers)
    response_text = response_text.replace("</s>", "").strip()

    # Add assistant response to conversation
    conversation_state.append({"role": "assistant", "content": response_text})

    # Update the chat history for Gradio display
    chat_history.append((message, response_text))

    return chat_history, conversation_state

# Create the Gradio interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 📊 Financial Markets Tutor & Adviser 📊")
    gr.Markdown("Ask questions about financial markets, investing strategies, and more!")

    chatbot = gr.Chatbot(
        value=[],
        show_label=False,
        height=500,
        bubble_full_width=False,
        show_copy_button=True,
    )

    conversation_state = gr.State([])

    with gr.Row():
        message = gr.Textbox(
            show_label=False,
            placeholder="Type your question here...",
            scale=5
        )
        submit = gr.Button("Send", scale=1)

    with gr.Accordion("About this Tutor", open=False):
        gr.Markdown("""
        ### Tips for using the Financial Tutor:
        - Start by sharing your experience level (beginner, intermediate, advanced)
        - Ask specific questions about markets, instruments, or strategies
        - For investment advice, mention your budget and risk tolerance
        - Be patient - the model may take a moment to respond on first use
        """)

    # Initialize the chat with a greeting
    def initialize_chat():
        initial_message = "Hello! I'm your financial markets tutor. Before we begin, could you tell me your current level of knowledge about financial markets? Are you a beginner, intermediate, or advanced?"
        return [(None, initial_message)], [{"role": "assistant", "content": initial_message}]

    # Functions for handling user input
    def user_input(message, chat_history, conversation_state):
        if message == "":
            return chat_history, conversation_state, gr.update()
        return respond(message, chat_history, conversation_state) + (gr.update(value=""),)

    # Wire up the interface components
    message.submit(user_input, [message, chatbot, conversation_state], [chatbot, conversation_state, message])
    submit.click(user_input, [message, chatbot, conversation_state], [chatbot, conversation_state, message])

    # Initialize the chat on page load
    demo.load(initialize_chat, [], [chatbot, conversation_state])

# Launch the app for HF Spaces
demo.launch()

<ipython-input-3-39a60cc45718>:142: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
<ipython-input-3-39a60cc45718>:142: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dc1b115386079d1f89.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
